In [1]:
import numpy as np

In [2]:

# type
A=1 # shen
B=2 # min
L=3 # lang
Y=4 # yu yan jia
S=5 # shou wei
H=6 # lie ren

# status
U = 0 # unknown
J = 1 # jin shui

In [3]:
def kill(people, mask, turn):
    idxs = np.where(people!=L)[0]
    idx = np.random.choice(idxs)
    if (turn%2==1 and people[idx]!=S) or (turn%2==0 and people[idx]!=Y):
        flag_revenge = people[idx] == H
        
        people = np.concatenate([people[:idx], people[idx+1:]])
        mask = np.concatenate([mask[:idx], mask[idx+1:]])
        
        if flag_revenge:
            idxs = np.where(mask!=J)[0]
            idx = np.random.choice(idxs)
            people = np.concatenate([people[:idx], people[idx+1:]])
            mask = np.concatenate([mask[:idx], mask[idx+1:]])    
    return people, mask

In [4]:
def vote(people, mask):
    idxs = np.where(mask!=J)[0]
    idx = np.random.choice(idxs)
    people = np.concatenate([people[:idx], people[idx+1:]])
    mask = np.concatenate([mask[:idx], mask[idx+1:]])
    return people, mask

In [10]:
def check(people, mask):
    idxs = np.where(np.logical_and(mask!=J, people!=Y))[0]
    idx = np.random.choice(idxs)
    if people[idx] == L:
        people = np.concatenate([people[:idx], people[idx+1:]])
        mask = np.concatenate([mask[:idx], mask[idx+1:]])
        flag = True
    else:
        mask[idx] = J
        flag = False
    return people, flag, mask

In [6]:
def min_all_killed(people):
    return A not in people or (H not in people and Y not in people and S not in people)

In [7]:
# monte carlo
import tqdm

In [13]:
cnt_L = 0
cnt_M = 0
for i in tqdm.tqdm(range(100000)):
    #people = np.array([A,A,A,A,B,B,B,B,L,L,L,L])
    people = np.array([A,A,A,A,H,H,S,Y,L,L,L,L])
    mask   = np.array([U,U,U,U,U,U,U,U,U,U,U,U])
    
    for turn in range(1, 100):
        if min_all_killed(people):
            cnt_L += 1
            break
        if L not in people:
            cnt_M += 1
            break
        if turn > 1:
            people, mask = kill(people, mask, turn)
        
        if min_all_killed(people):
            cnt_L += 1
            break
        if L not in people:
            cnt_M += 1
            break
        
        successfully_checked = False
        if Y in people:
            people, successfully_checked, mask = check(people, mask)
            if min_all_killed(people):
                cnt_L += 1
                break
            if L not in people:
                cnt_M += 1
                break
        if not successfully_checked:
            people, mask = vote(people, mask)
cnt_L, cnt_M

100%|████████████████████████████████████████████████████████████████| 100000/100000 [00:30<00:00, 3266.96it/s]


(59460, 40540)